# AI-Based Automated Grading System for Lecture Summaries

This notebook implements a fully automated, explainable, and reproducible AI grading system that evaluates student-written summaries by comparing them to lecture transcripts.

## Configuration
Set your roll number below to generate the correct output filename.


In [ ]:
# Configuration: Set your roll number here
ROLL_NUMBER = "22bec021"  # Replace with your actual roll number

# Input file paths
LECTURE_TRANSCRIPT_PATH = "C:\Users\jallu\Downloads\7Aug2025 T+S(Agentic AI)\Agentic AI Class - 2025_08_07 18_10 GMT+05_30 - Transcript.docx"
STUDENT_SUMMARIES_PATH = "C:\Users\jallu\Downloads\7Aug2025 T+S(Agentic AI)\Attendance + Summary 1.xlsx"

# Output file path
OUTPUT_FILE_PATH = f"Gradedresults{ROLL_NUMBER}.xlsx"


## 1. Install and Import Required Libraries


In [ ]:
# Install required packages if not already installed
import subprocess
import sys

def install_package(package, import_name=None):
    """
    Install a package if not already installed.
    
    Args:
        package (str): Package name for pip install
        import_name (str): Name to use for import (if different from package name)
    """
    if import_name is None:
        # Map package names to their import names
        import_map = {
            "python-docx": "docx",
            "scikit-learn": "sklearn"
        }
        import_name = import_map.get(package, package.replace("-", "_"))
    
    try:
        __import__(import_name)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package], 
                            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Install required packages
packages = [
    ("python-docx", "docx"),
    ("pandas", "pandas"),
    ("openpyxl", "openpyxl"),
    ("sentence-transformers", "sentence_transformers"),
    ("numpy", "numpy"),
    ("scikit-learn", "sklearn")
]

for package, import_name in packages:
    install_package(package, import_name)

print("All required packages installed successfully!")


In [ ]:
# Import all required libraries
import os
import pandas as pd
import numpy as np
from docx import Document
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")


## 2. Load Lecture Transcript from DOCX


In [ ]:
def load_lecture_transcript(docx_path):
    """
    Load and extract text from a DOCX file containing the lecture transcript.
    
    Args:
        docx_path (str): Path to the DOCX file
    
    Returns:
        str: Full text content of the lecture transcript
    """
    try:
        if not os.path.exists(docx_path):
            raise FileNotFoundError(f"Lecture transcript file not found: {docx_path}")
        
        doc = Document(docx_path)
        full_text = []
        
        # Extract text from all paragraphs
        for paragraph in doc.paragraphs:
            if paragraph.text.strip():  # Skip empty paragraphs
                full_text.append(paragraph.text.strip())
        
        # Join all paragraphs with newlines
        transcript = "\n".join(full_text)
        
        # Validate that transcript is not empty
        if len(transcript.strip()) < 50:
            raise ValueError("Lecture transcript appears to be empty or too short. Please check the file.")
        
        print(f"Successfully loaded lecture transcript ({len(transcript)} characters)")
        return transcript
    except Exception as e:
        print(f"Error loading lecture transcript: {e}")
        raise

# Load the lecture transcript
lecture_transcript = load_lecture_transcript(LECTURE_TRANSCRIPT_PATH)
print(f"\nFirst 500 characters of transcript:\n{lecture_transcript[:500]}...")


## 3. Load Student Summaries from Excel


In [ ]:
def load_student_summaries(excel_path):
    """
    Load student summaries from an Excel file.
    
    Args:
        excel_path (str): Path to the Excel file
    
    Returns:
        pd.DataFrame: DataFrame containing student information and summaries
    """
    try:
        if not os.path.exists(excel_path):
            raise FileNotFoundError(f"Student summaries file not found: {excel_path}")
        
        df = pd.read_excel(excel_path)
        
        # Validate that DataFrame is not empty
        if len(df) == 0:
            raise ValueError("Student summaries file is empty. Please check the file.")
        
        # Expected columns: Email Address, Name of the Student, Roll Number, Institute, Summary
        print(f"Loaded {len(df)} student summaries")
        print(f"\nColumns found: {list(df.columns)}")
        print(f"\nFirst few rows:")
        print(df.head())
        
        return df
    except Exception as e:
        print(f"Error loading student summaries: {e}")
        raise

# Load student summaries
student_df = load_student_summaries(STUDENT_SUMMARIES_PATH)


## 4. Initialize AI Models for Evaluation


In [ ]:
# Initialize Sentence Transformer model for semantic similarity
# Using a pre-trained model that works well for semantic similarity tasks
print("Loading Sentence Transformer model...")
print("Note: This may take a few minutes on first run as the model downloads (~80MB)")
try:
    model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight and effective model
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure you have an internet connection for the first run.")
    raise


## 5. Extract Key Points from Lecture Transcript


In [ ]:
def extract_key_points(transcript, max_sentences=50):
    """
    Extract key points from the lecture transcript by splitting into sentences
    and identifying important sections.
    
    Args:
        transcript (str): Full lecture transcript text
        max_sentences (int): Maximum number of sentences to consider
    
    Returns:
        list: List of key sentences/points from the transcript
    """
    # Split transcript into sentences
    sentences = transcript.replace('\n', ' ').split('.')
    sentences = [s.strip() for s in sentences if len(s.strip()) > 20]  # Filter very short sentences
    
    # Take a representative sample of sentences (beginning, middle, end)
    if len(sentences) > max_sentences:
        # Sample from different parts of the transcript
        step = len(sentences) // max_sentences
        key_sentences = sentences[::step][:max_sentences]
    else:
        key_sentences = sentences
    
    return key_sentences

# Extract key points from lecture transcript
key_points = extract_key_points(lecture_transcript)
print(f"Extracted {len(key_points)} key points from the lecture transcript")
print(f"\nSample key points:\n{key_points[:3]}")


## 6. Evaluation Function - Hybrid Approach (Embeddings + Rule-Based Scoring)


In [ ]:
def calculate_semantic_similarity(summary, reference_text, model):
    """
    Calculate semantic similarity between summary and reference text using embeddings.
    
    Args:
        summary (str): Student's summary text
        reference_text (str): Reference text (lecture transcript or key points)
        model: SentenceTransformer model
    
    Returns:
        float: Similarity score between 0 and 1
    """
    # Truncate very long texts to avoid memory issues (max 5000 chars for reference)
    if len(reference_text) > 5000:
        reference_text = reference_text[:5000]
    
    # Truncate very long summaries (max 2000 chars)
    if len(summary) > 2000:
        summary = summary[:2000]
    
    # Generate embeddings
    summary_embedding = model.encode(summary, convert_to_numpy=True, show_progress_bar=False)
    reference_embedding = model.encode(reference_text, convert_to_numpy=True, show_progress_bar=False)
    
    # Calculate cosine similarity
    similarity = cosine_similarity([summary_embedding], [reference_embedding])[0][0]
    
    return float(similarity)

def calculate_coverage_score(summary, key_points, model):
    """
    Calculate how well the summary covers key points from the lecture.
    
    Args:
        summary (str): Student's summary text
        key_points (list): List of key points from the lecture
        model: SentenceTransformer model
    
    Returns:
        float: Coverage score between 0 and 1
    """
    if not key_points or not summary:
        return 0.0
    
    # Truncate very long summaries (max 2000 chars)
    if len(summary) > 2000:
        summary = summary[:2000]
    
    # Calculate similarity between summary and each key point
    summary_embedding = model.encode(summary, convert_to_numpy=True, show_progress_bar=False)
    
    similarities = []
    for point in key_points:
        if len(point.strip()) > 10:  # Only consider substantial points
            # Truncate very long key points
            point_text = point[:500] if len(point) > 500 else point
            point_embedding = model.encode(point_text, convert_to_numpy=True, show_progress_bar=False)
            similarity = cosine_similarity([summary_embedding], [point_embedding])[0][0]
            similarities.append(similarity)
    
    # Coverage is the average of top similarities (how many key points are covered)
    if similarities:
        # Consider a key point "covered" if similarity > 0.3
        covered_points = sum(1 for s in similarities if s > 0.3)
        coverage_score = covered_points / len(similarities)
        return min(coverage_score, 1.0)
    
    return 0.0

def calculate_clarity_score(summary):
    """
    Calculate clarity and coherence score based on text characteristics.
    
    Args:
        summary (str): Student's summary text
    
    Returns:
        float: Clarity score between 0 and 1
    """
    if not summary or len(summary.strip()) < 10:
        return 0.0
    
    # Basic heuristics for clarity
    word_count = len(summary.split())
    sentence_count = len([s for s in summary.split('.') if s.strip()])
    
    # Check for reasonable sentence length (not too short, not too long)
    avg_sentence_length = word_count / max(sentence_count, 1)
    
    # Score based on:
    # - Has reasonable length (not too short, not too verbose)
    # - Has multiple sentences (shows structure)
    # - Average sentence length is reasonable (10-25 words is good)
    
    length_score = min(1.0, word_count / 100)  # Prefer summaries with at least 100 words
    structure_score = min(1.0, sentence_count / 5)  # Prefer at least 5 sentences
    sentence_quality = 1.0 if 10 <= avg_sentence_length <= 30 else 0.7
    
    # Combined clarity score
    clarity = (length_score * 0.3 + structure_score * 0.3 + sentence_quality * 0.4)
    
    return min(clarity, 1.0)

def check_grammar_basic(summary):
    """
    Basic grammar check using simple heuristics.
    
    Args:
        summary (str): Student's summary text
    
    Returns:
        float: Grammar score between 0 and 1
    """
    if not summary or len(summary.strip()) < 10:
        return 0.0
    
    # Basic checks:
    # - Proper capitalization at sentence start
    # - Sentence ending punctuation
    # - No excessive repeated characters
    
    sentences = [s.strip() for s in summary.split('.') if s.strip()]
    if not sentences:
        return 0.0
    
    proper_caps = sum(1 for s in sentences if s and s[0].isupper()) / len(sentences)
    
    # Check for excessive repetition (e.g., "aaaa")
    has_repetition = any(len(set(word)) == 1 and len(word) > 3 for word in summary.split())
    repetition_penalty = 0.0 if has_repetition else 1.0
    
    grammar_score = (proper_caps * 0.7 + repetition_penalty * 0.3)
    
    return grammar_score

print("Evaluation functions defined successfully!")


## 7. Main Evaluation Function


In [ ]:
def generate_explanation(score, coverage, similarity, clarity, completeness, grammar, summary, word_count):
    """
    Generate a 2-3 sentence explanation justifying the score.
    
    Args:
        score (float): Final score out of 10
        coverage (float): Coverage score
        similarity (float): Semantic similarity score
        clarity (float): Clarity score
        completeness (float): Completeness score
        grammar (float): Grammar score
        summary (str): The summary text
        word_count (int): Word count of summary
    
    Returns:
        str: Explanation text
    """
    strengths = []
    weaknesses = []
    
    # Identify strengths
    if coverage > 0.7:
        strengths.append("covers most key lecture points")
    elif coverage > 0.4:
        strengths.append("covers some key lecture points")
    
    if similarity > 0.6:
        strengths.append("maintains good semantic alignment with the lecture content")
    
    if clarity > 0.7:
        strengths.append("demonstrates clear and coherent writing")
    
    if completeness > 0.7:
        strengths.append("provides comprehensive coverage of major topics")
    
    if grammar > 0.8:
        strengths.append("shows good grammatical structure")
    
    # Identify weaknesses
    if coverage < 0.4:
        weaknesses.append("omits many key lecture points")
    elif coverage < 0.6:
        weaknesses.append("misses some important lecture points")
    
    if similarity < 0.4:
        weaknesses.append("shows limited alignment with lecture content")
    
    if clarity < 0.5:
        weaknesses.append("lacks clarity and coherent structure")
    
    if word_count < 50:
        weaknesses.append("is too brief and lacks detail")
    elif word_count > 500:
        weaknesses.append("is overly verbose and could be more concise")
    
    if grammar < 0.6:
        weaknesses.append("contains grammatical issues")
    
    # Construct explanation
    explanation_parts = []
    
    if strengths:
        explanation_parts.append(f"The summary {', '.join(strengths[:2])}.")
    
    if weaknesses:
        explanation_parts.append(f"However, it {', '.join(weaknesses[:2])}.")
    
    if not explanation_parts:
        if score >= 7:
            explanation_parts.append("The summary provides a solid overview of the lecture with good coverage and clarity.")
        elif score >= 4:
            explanation_parts.append("The summary demonstrates moderate understanding but could benefit from more comprehensive coverage.")
        else:
            explanation_parts.append("The summary requires significant improvement in coverage, clarity, and alignment with lecture content.")
    
    # Add score context
    if score >= 8:
        explanation_parts.append("Overall, this represents a strong summary that effectively captures the lecture's main points.")
    elif score >= 6:
        explanation_parts.append("The summary meets basic requirements but has room for improvement in depth and completeness.")
    elif score >= 4:
        explanation_parts.append("The summary shows partial understanding but lacks sufficient detail and coverage.")
    else:
        explanation_parts.append("The summary needs substantial revision to adequately represent the lecture content.")
    
    return " ".join(explanation_parts)

def evaluate_summary(summary, lecture_transcript, key_points, model):
    """
    Comprehensive evaluation of a student summary against the lecture transcript.
    
    Args:
        summary (str): Student's summary text
        lecture_transcript (str): Full lecture transcript
        key_points (list): List of key points from the lecture
        model: SentenceTransformer model for embeddings
    
    Returns:
        tuple: (score out of 10, explanation string)
    """
    if not summary or len(summary.strip()) < 10:
        return 0.0, "Summary is too short or empty. No meaningful content provided."
    
    # Calculate individual component scores (all between 0 and 1)
    
    # 1. Semantic Similarity (how well summary matches overall transcript)
    semantic_sim = calculate_semantic_similarity(summary, lecture_transcript[:5000], model)  # Use first 5000 chars for efficiency
    
    # 2. Coverage of main lecture points
    coverage_score = calculate_coverage_score(summary, key_points, model)
    
    # 3. Clarity and coherence
    clarity_score = calculate_clarity_score(summary)
    
    # 4. Grammar (basic check)
    grammar_score = check_grammar_basic(summary)
    
    # 5. Completeness (based on length and coverage)
    word_count = len(summary.split())
    completeness_score = min(1.0, (coverage_score * 0.7 + min(1.0, word_count / 200) * 0.3))
    
    # Weighted combination of scores
    # Coverage: 30%, Semantic Similarity: 25%, Clarity: 20%, Completeness: 15%, Grammar: 10%
    final_score = (
        coverage_score * 0.30 +
        semantic_sim * 0.25 +
        clarity_score * 0.20 +
        completeness_score * 0.15 +
        grammar_score * 0.10
    )
    
    # Convert to 0-10 scale
    score_0_10 = final_score * 10
    
    # Generate explanation
    explanation = generate_explanation(
        score_0_10, coverage_score, semantic_sim, clarity_score, 
        completeness_score, grammar_score, summary, word_count
    )
    
    return round(score_0_10, 2), explanation

print("Main evaluation function defined successfully!")


## 8. Process All Student Summaries


In [10]:
def process_all_summaries(student_df, lecture_transcript, key_points, model):
    """
    Process all student summaries and generate scores and explanations.
    
    Args:
        student_df (pd.DataFrame): DataFrame with student information and summaries
        lecture_transcript (str): Full lecture transcript
        key_points (list): List of key points from the lecture
        model: SentenceTransformer model
    
    Returns:
        pd.DataFrame: DataFrame with all original columns plus Score and Explanation
    """
    # Validate inputs
    if student_df is None or len(student_df) == 0:
        raise ValueError("Student DataFrame is empty or None")
    
    if not lecture_transcript or len(lecture_transcript.strip()) < 50:
        raise ValueError("Lecture transcript is empty or too short")
    
    if not key_points or len(key_points) == 0:
        raise ValueError("No key points extracted from lecture transcript")
    
    results = []
    
    # Determine the summary column name (handle variations)
    summary_col = None
    possible_names = ['Summary', 'summary', 'Summary Text', 'Student Summary']
    for name in possible_names:
        if name in student_df.columns:
            summary_col = name
            break
    
    if summary_col is None:
        # Try to find a column that might contain summaries
        for col in student_df.columns:
            if 'summary' in col.lower():
                summary_col = col
                break
    
    if summary_col is None:
        raise ValueError(f"Could not find summary column. Available columns: {list(student_df.columns)}")
    
    print(f"Using column '{summary_col}' for summaries")
    print(f"\nProcessing {len(student_df)} summaries...")
    
    for idx, row in student_df.iterrows():
        # Handle NaN and None values
        summary = str(row[summary_col]) if pd.notna(row[summary_col]) else ""
        summary = summary.strip() if summary else ""
        
        print(f"Processing student {idx + 1}/{len(student_df)}...", end="\r")
        
        try:
            # Evaluate the summary
            score, explanation = evaluate_summary(summary, lecture_transcript, key_points, model)
        except Exception as e:
            print(f"\nWarning: Error evaluating summary for student {idx + 1}: {e}")
            score = 0.0
            explanation = f"Error during evaluation: {str(e)}"
        
        # Create result row - preserve all original columns
        result_row = row.to_dict()
        result_row['Numerical Score'] = score
        result_row['Short explanation'] = explanation
        
        results.append(result_row)
    
    print(f"\n\nCompleted processing all {len(student_df)} summaries!")
    
    # Create results DataFrame
    if not results:
        raise ValueError("No results generated. Please check your input data.")
    
    results_df = pd.DataFrame(results)
    
    return results_df

# Process all summaries
print("Starting batch processing of all student summaries...")
graded_results = process_all_summaries(student_df, lecture_transcript, key_points, model)


Processing student 82/82...

Completed processing all 82 summaries!


## 9. Prepare Output DataFrame with Required Columns


In [11]:
def prepare_output_dataframe(graded_results):
    """
    Prepare the final output DataFrame with all required columns in the correct order.
    
    Args:
        graded_results (pd.DataFrame): DataFrame with graded results
    
    Returns:
        pd.DataFrame: Formatted output DataFrame
    """
    # Validate that required columns exist
    if 'Numerical Score' not in graded_results.columns:
        raise ValueError("Missing 'Numerical Score' column in graded results")
    if 'Short explanation' not in graded_results.columns:
        raise ValueError("Missing 'Short explanation' column in graded results")
    
    # Map column names (handle variations)
    column_mapping = {
        'Email Address': ['Email Address', 'Email', 'email', 'Email address', 'Email Address'],
        'Name': ['Name of the Student', 'Name', 'Student Name', 'name', 'Name of Student'],
        'Roll Number': ['Roll Number', 'Roll No', 'Roll', 'roll number', 'RollNumber'],
        'Institute': ['Institute', 'institute', 'Institution', 'Institution Name'],
        'Original Summary': None,  # Will be determined from summary column
        'Numerical Score': 'Numerical Score',
        'Short explanation': 'Short explanation'
    }
    
    output_data = {}
    
    # Find and map columns
    for target_col, possible_names in column_mapping.items():
        if possible_names is None:
            # Special handling for Original Summary
            summary_col = None
            for col in graded_results.columns:
                if 'summary' in col.lower() and col not in ['Short explanation', 'Short Explanation']:
                    summary_col = col
                    break
            if summary_col:
                output_data[target_col] = graded_results[summary_col].fillna("")
            else:
                # Create empty column if summary column not found
                output_data[target_col] = [""] * len(graded_results)
        else:
            found = False
            for possible_name in possible_names:
                if possible_name in graded_results.columns:
                    # Fill NaN values with empty string
                    output_data[target_col] = graded_results[possible_name].fillna("")
                    found = True
                    break
            if not found:
                # If column not found, create empty column
                output_data[target_col] = [""] * len(graded_results)
    
    # Ensure all columns have the same length
    expected_length = len(graded_results)
    for key, value in output_data.items():
        if isinstance(value, pd.Series):
            if len(value) != expected_length:
                raise ValueError(f"Column '{key}' has incorrect length: {len(value)} != {expected_length}")
        elif isinstance(value, list):
            if len(value) != expected_length:
                output_data[key] = [""] * expected_length
    
    # Create output DataFrame with required columns in order
    output_df = pd.DataFrame({
        'Email Address': output_data.get('Email Address', [""] * expected_length),
        'Name': output_data.get('Name', [""] * expected_length),
        'Roll Number': output_data.get('Roll Number', [""] * expected_length),
        'Institute': output_data.get('Institute', [""] * expected_length),
        'Original Summary': output_data.get('Original Summary', [""] * expected_length),
        'Numerical Score': graded_results['Numerical Score'],
        'Short explanation': graded_results['Short explanation']
    })
    
    # Validate output DataFrame
    if len(output_df) == 0:
        raise ValueError("Output DataFrame is empty")
    
    return output_df

# Prepare output DataFrame
output_df = prepare_output_dataframe(graded_results)

print("Output DataFrame prepared!")
print(f"\nColumns: {list(output_df.columns)}")
print(f"\nShape: {output_df.shape}")
print(f"\nFirst few rows:")
print(output_df.head())


Output DataFrame prepared!

Columns: ['Email Address', 'Name', 'Roll Number', 'Institute', 'Original Summary', 'Numerical Score', 'Short explanation']

Shape: (82, 7)

First few rows:
            Email Address                    Name Roll Number     Institute  \
0   cs22b1027@iiitr.ac.in                 Deepthi              IIIT Raichur   
1  23bds014@iiitdwd.ac.in            Bongu Ashish              IIIT Dharwad   
2   cs22b1036@iiitr.ac.in  Mudavath Srinivas Naik              IIIT Raichur   
3   cs22b1007@iiitr.ac.in          Aman Chaurasia              IIIT Raichur   
4   cs22b1032@iiitr.ac.in            Krishu Patel              IIIT Raichur   

                                    Original Summary  Numerical Score  \
0  Learned about neural networks in AI generative...             2.44   
1                                         Insightful             2.03   
2  In today’s class, I learned about the Med Agen...             3.71   
3  In this class, I learned the basics to advance

## 10. Export Results to Excel


In [12]:
def export_to_excel(output_df, output_path):
    """
    Export the graded results to an Excel file.
    
    Args:
        output_df (pd.DataFrame): DataFrame with graded results
        output_path (str): Path to output Excel file
    """
    try:
        # Validate DataFrame before export
        if output_df is None or len(output_df) == 0:
            raise ValueError("Cannot export empty DataFrame")
        
        # Check if required columns exist
        required_cols = ['Email Address', 'Name', 'Roll Number', 'Institute', 
                        'Original Summary', 'Numerical Score', 'Short explanation']
        missing_cols = [col for col in required_cols if col not in output_df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")
        
        # Ensure output directory exists
        output_dir = os.path.dirname(output_path) if os.path.dirname(output_path) else "."
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir, exist_ok=True)
        
        # Export to Excel
        output_df.to_excel(output_path, index=False, engine='openpyxl')
        
        # Verify file was created
        if not os.path.exists(output_path):
            raise FileNotFoundError(f"Output file was not created: {output_path}")
        
        print(f"\nSuccessfully exported results to {output_path}")
        print(f"Total students graded: {len(output_df)}")
        print(f"\nScore statistics:")
        print(f"  Mean: {output_df['Numerical Score'].mean():.2f}")
        print(f"  Min: {output_df['Numerical Score'].min():.2f}")
        print(f"  Max: {output_df['Numerical Score'].max():.2f}")
        print(f"  Std: {output_df['Numerical Score'].std():.2f}")
        
        # Show file size
        file_size = os.path.getsize(output_path) / 1024  # KB
        print(f"  File size: {file_size:.2f} KB")
        
    except Exception as e:
        print(f"Error exporting to Excel: {e}")
        raise

# Export results
export_to_excel(output_df, OUTPUT_FILE_PATH)

print("\n" + "="*50)
print("GRADING COMPLETE!")
print("="*50)
print(f"Output file: {OUTPUT_FILE_PATH}")
print(f"File location: {os.path.abspath(OUTPUT_FILE_PATH)}")



Successfully exported results to Gradedresults22bec021.xlsx
Total students graded: 82

Score statistics:
  Mean: 3.83
  Min: 2.03
  Max: 4.81
  Std: 0.59
  File size: 23.05 KB

GRADING COMPLETE!
Output file: Gradedresults22bec021.xlsx
File location: c:\Users\jallu\OneDrive\Desktop\Quiz\Gradedresults22bec021.xlsx


## Summary

This notebook implements a fully automated AI-based grading system that:

1. **Loads** the lecture transcript from DOCX format
2. **Loads** student summaries from Excel format
3. **Evaluates** each summary using a hybrid approach:
   - Semantic similarity using Sentence Transformers
   - Coverage analysis of key lecture points
   - Clarity and coherence assessment
   - Completeness evaluation
   - Basic grammar checking
4. **Generates** scores (0-10) with detailed explanations
5. **Exports** results to Excel with all required columns

The system is fully automated, explainable, and reproducible. All evaluation logic is contained within this notebook.
